**Exercise 10 - Transfer Learning**

---

In this exercise you will work with transfer learning on image classification. In this task you will use the ResNet to classify images, however, differently from the other tasks, you will not use a huge dataset, but a small "handmade" dataset. In the exercise directory you have a *data* directory with some pictures taken with a smartphone, you can use it or if you prefer, use your own cellphone to take your own pictures. Since you will have to label it, there will no advantage of using this or your data. Your task will be, first, define the classes and label the images, then, build the model to classify those images, and last, use the pretrained ResNet weights and compare the results.


![Transfer learning](mlrcv/notebook_images/transferlearn.png)



**10.1 Labeling data \[3\]**

---

Given the selected dataset (the images provided, or the one that you have collected) you should label it. Inside the *data* directory there are two other directories *train_img* and *val_img* with the train and validation images. If you collect your own data you should mantain these directories structure. After defining the data that you will work with, you can start labeling it.

First you need to check and define the classes present on dataset, this definition should be mapped on the *class_to_lbl* dictionary inside the file *mlrcv/data_map.py*. The mapping should be on the format:

```
class_to_lbl = {
    0: "dog",
    1: "cat",
    ...
}
```

Then, you should fill the other two dictionary inside *mlrcv/data_map.py*, *train_annotation* and *val_annotation*, with the image-wise annotation, giving to each image a unique label. Following the format:

```
train_annotation = {
    "xxxx.jpg": 6,
    "xxxy.jpg": 0,
    ...
}

val_annotation = {
    "zzzz.jpg": 0,
    "zzzw.jpg": 4,
    ...
}
```

After filling the dictionaries you check if it's correctly mapped by running the cell below:

In [ ]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import numpy as np
import torchvision
from torchvision import transforms, models
import matplotlib.pyplot as plt
from mlrcv.data_map import class_to_lbl, train_annotation, val_annotation
from mlrcv.dataloader import ImageLoader, transform_train, transform_val
from mlrcv.trainer import ClassificationTrainer, freeze_net_params
from mlrcv.model_loader import build_classifier
from PIL import Image
%load_ext autoreload
%autoreload 2

!mkdir train_logs
!mkdir checkpoint

In [ ]:
ind = np.random.choice(len(train_annotation.keys()), 1)[0]
dict_key = list(train_annotation.keys())[ind]

img = Image.open(f'data/train_img/{dict_key}')
plt.imshow(img)
plt.xlabel(f'Image {dict_key} label {train_annotation[dict_key]} class {class_to_lbl[train_annotation[dict_key]]}')
plt.show()

After generating the annotation and class dictionaries, by running the cell above you should see one random image from train dataset with it's label written on the $x$ axis, so you can check if your annotations are correctly mapped.

**10.2 Bulding the dataloader \[3\]**

---

After labelling the data you should load it properly to use it on pytorch. Your next task is to implement the *DataLoader*, torchvision provides many datasets DataLoaders already implemented, but usually it's necessary to implement your own DataLoader to deal with your data and task specifications. You should implement the *load_images* function from *ImageLoader* class inside *mlrcv/dataloader.py*. You should load the image data and it's annotation, create a dictionary with:

```
{
    "img": Image, #PIL Image
    "label": label, #label as int
}
```

Then, append it to the *images_annotation* list class attribute.

In [ ]:
test_data = ImageLoader('./data/train_img', train_annotation, transform=transform_train)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=4)

test_iter = iter(test_loader)
data = test_iter.next()

img = data['img'][0].permute(1,2,0)
plt.imshow(img)
plt.xlabel(f'Label {data["label"].item()} class {class_to_lbl[data["label"].item()]}')
plt.show()

As before, if everything works you should see above an image from the dataset with it's label and class on $x$ axis.

**10.3 Bulding the model \[3\]**

---

The ResNet model can be used directly from the *torchvision* implementation, however, in their implementation the network is built for the ImageNet dataset, which means that the last layer outputs 1000 values, which is the number of classes on ImageNet dataset. To use it on a different dataset you should remove the last layer and append to it your own final layers (see figure 1). In this task you should implement the function *build_resnet_model* to build the model to classify the dataset that you have labeled.


In [ ]:
num_class = 13

resnet = models.resnet18()
# the resnet output has shape [batch_size, 512]

# overwrite the classifier
resnet.fc = build_classifier(num_class=13)
output = resnet(data['img'])

assert output.shape[-1] == num_class,\
    f"[num_class({num_class}) != out_feats({output.shape[-1]})]: output features num should be equal to the num_class"

print(resnet.fc)


You should see above the classifier that you have built with the layers and features sizes that you defined.

Before start the training, start tensorboard. The logs will be saved in the center_logs/, so you can start tensorboard inside the notebook, or through terminal by running tensorboard --logdir=center_logs which then can be visualized on the browser acessing localhost:6006.

In [ ]:
# setup tensorboard
%load_ext tensorboard
%tensorboard --logdir train_logs

Then, you can train this network using your data:

In [ ]:
train_data = ImageLoader('./data/train_img', train_annotation, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True, num_workers=4)

val_data = ImageLoader('./data/val_img', val_annotation, transform=transform_train)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=num_class, num_workers=4)

lr = 1e-3
epochs = 100

In [ ]:
resnet = models.resnet18()
# overwrite the classifier
resnet.fc = build_classifier(num_class=13)

classifier = ClassificationTrainer(resnet, torch.nn.CrossEntropyLoss(), train_loader, val_loader, class_to_lbl, lr, epochs, log_name='no_transfer_learning')
trainer = pl.Trainer(max_epochs=epochs, check_val_every_n_epoch=10)
trainer.fit(classifier)

On tensorboard you can follow the training and check your network performance on this small dataset. It's possible to see an overfitting on the training data, the accuracy increase while the loss decrease, however the accuracy on the validation set is way lower than the training accuracy. Due to the low number of samples the network can't find general representative features to correctly classify the images. You can try changing the classifier the architecture that you have defined on *build_resnet_model* and try to improve it's performance.

**10.4 Transfer learning \[1\]**

---

From the previous task you should have noticed that even with different architectures for the last layers the performance on this small dataset can not be increased significantly, leading to overfit on training data and poor performance on validation data. This is a problem with small datasets, without enough data it's hard for the network to generalize a good solution even for the training data. In this last task you will use transfer learning to improve your model performance.

Alongside the implemented model, *torchvision* also provide the network trained weights. So you can load it, and do the training on your small dataset using the weights learned from a bigger dataset, in this case, ImageNet. The advantage of using the pretrained weights is to start the network training with weights good enough to extract representative features, and then finetune it to the current dataset specific characteristics.

In [ ]:
resnet = models.resnet18(pretrained=True)
# overwrite the classifier
resnet.fc = build_classifier(num_class=13)

classifier = ClassificationTrainer(resnet, torch.nn.CrossEntropyLoss(), train_loader, val_loader, class_to_lbl, lr, epochs, log_name='transfer_learning_train_finetune')
trainer = pl.Trainer(max_epochs=epochs, check_val_every_n_epoch=10)
trainer.fit(classifier)

From the logs of the previous cell you can compare on tensorboard the results from training the network with and without transfer learning. From this comparison it's possible to see that, by using the ImageNet pretrained weights the train and validation accuracy are much higher than when training the network without the pretrained weights. Also, it converge faster to the final solution, since from the beginning the network can already extract representative/general features from the image. However, at some point the performance may start decreasing, this can happen because the network starts losing generalization and starts also to overfit on your data particularities.

Other possibility is to also freeze the pretrained weights, training only the final classifier. We do not update the ImageNet pretrained weights and update only the classifier during training.

In [ ]:
resnet = models.resnet18(pretrained=True)
resnet = freeze_net_params(resnet)
# overwrite the classifier
resnet.fc = build_classifier(num_class=13)

classifier = ClassificationTrainer(resnet, torch.nn.CrossEntropyLoss(), train_loader, val_loader, class_to_lbl, lr, epochs, log_name='transfer_learning_train_freezed')
trainer = pl.Trainer(max_epochs=epochs, check_val_every_n_epoch=10)
trainer.fit(classifier)

By freezing the pretrained weights it may improve even more the performance of the network (you can compare on tensorboard). With a small dataset, if we update the pretrained weights it may be decrease the performance since this dataset is not representative enough, and this poor dataset can "harm" the network weights making the network lose some generalization and performance.

Using the pretrained network and freezing it's weights brings a better performance, achieving high accuracy on both training and validation sets.